## Deeplearning

In [61]:
# Import necessary libraries
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [63]:
# Load the dataset
df = pd.read_csv('Symptom2Disease.csv')
df.head()

,Unnamed: 0,label,text
0,0,Psoriasis,I have been experiencing a skin rash on my arm...
1,1,Psoriasis,"My skin has been peeling, especially on my kne..."
2,2,Psoriasis,I have been experiencing joint pain in my fing...
3,3,Psoriasis,"There is a silver like dusting on my skin, esp..."
4,4,Psoriasis,"My nails have small dents or pits in them, and..."


In [65]:
num_categories = df['label'].nunique()
print("The number of categories of diseases are:", num_categories)

The number of categories of diseases are: 24


In [66]:
# Split the dataset into input (text) and output (label) columns
text_data = df['text'].values
labels = df['label'].values

# Encode the labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)


In [97]:
# Split the dataset into training and testing sets
text_train, text_test, labels_train, labels_test = train_test_split(text_data, labels, test_size=0.2, random_state=42)

# Create a tokenizer and fit it on the training text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train)

# Convert the text data to sequences of tokens
sequences_train = tokenizer.texts_to_sequences(text_train)
sequences_test = tokenizer.texts_to_sequences(text_test)

# Pad the sequences to have the same length
max_sequence_length = max(len(seq) for seq in sequences_train)
sequences_train = pad_sequences(sequences_train, maxlen=max_sequence_length)
sequences_test = pad_sequences(sequences_test, maxlen=max_sequence_length)

# Determine the number of classes
num_classes = len(label_encoder.classes_)

# Define the deep learning model architecture
# model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dense(num_classes, activation='softmax')
# ])

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length),
    tf.keras.layers.Dropout(0.5),  # Add dropout layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),  # Add dropout layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add dropout layer
    tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),  # Add L2 regularization
    tf.keras.layers.Dropout(0.5),  # Add dropout layer
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Add dropout layer
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Train the model
model.fit(sequences_train, labels_train, epochs=50, batch_size=32)

Epoch 1/50
30/30 [==============================] - 1s 9ms/step - loss: 4.2173 - accuracy: 0.0312
Epoch 2/50
30/30 [==============================] - 0s 10ms/step - loss: 3.8045 - accuracy: 0.0542
Epoch 3/50
30/30 [==============================] - 0s 11ms/step - loss: 3.5560 - accuracy: 0.0500
Epoch 4/50
30/30 [==============================] - 0s 10ms/step - loss: 3.3875 - accuracy: 0.0688
Epoch 5/50
30/30 [==============================] - 0s 12ms/step - loss: 3.2822 - accuracy: 0.0625
Epoch 6/50
30/30 [==============================] - 0s 12ms/step - loss: 3.1820 - accuracy: 0.0969
Epoch 7/50
30/30 [==============================] - 0s 12ms/step - loss: 3.0183 - accuracy: 0.1385
Epoch 8/50
30/30 [==============================] - 0s 12ms/step - loss: 2.8011 - accuracy: 0.1740
Epoch 9/50
30/30 [==============================] - 0s 12ms/step - loss: 2.5516 - accuracy: 0.2396
Epoch 10/50
30/30 [==============================] - 0s 14ms/step - loss: 2.3874 - accuracy: 0.2542
Epoch 11/5

In [98]:
_, accuracy = model.evaluate(sequences_test, labels_test)
print("Accuracy:", accuracy)

8/8 [==============================] - 0s 1ms/step - loss: 0.6800 - accuracy: 0.8208
Accuracy: 0.8208333253860474


In [99]:
# Save the trained model
model.save('disease_model.h5')

In [100]:
# Create a function for prediction
def predict_disease(text):
    # Load the trained model
    model = tf.keras.models.load_model('disease_model.h5')

    # Preprocess the input text
    sequence = tokenizer.texts_to_sequences([text])
    sequence = pad_sequences(sequence, maxlen=max_sequence_length)

    # Make the prediction
    predicted_probs = model.predict(sequence)[0]
    predicted_class = tf.argmax(predicted_probs).numpy()
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    # Calculate the confidence score
    predicted_prob = predicted_probs[predicted_class]

    # Return the predicted class and confidence score
    return predicted_label, predicted_prob

In [101]:
# Test the predict_disease function
input_text = "My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms."
predicted_class = predict_disease(input_text)
print(predicted_class)

1/1 [==============================] - 0s 51ms/step
('Psoriasis', 0.9839914)
